In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'D:\\ML_Projects\\Violence-Reporter'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    prediction_model_path: Path
    model_checkpoints_path: Path
    model_history: Path
    model_evaluation_history: Path
    params_epochs: int
    params_batch_size: int
    params_image_height: list
    params_image_width: list
    params_test_size: float


In [5]:
from violenceReporter.constants import *
from violenceReporter.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_transformation.root_dir
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            prediction_model_path= Path(training.prediction_model_path),
            model_checkpoints_path= Path(training.model_checkpoints_path),
            model_evaluation_history= Path(training.model_evaluation_history),
            model_history= Path(training.model_history),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_height=params.IMAGE_HEIGHT,
            params_image_width=params.IMAGE_WIDTH,
            params_test_size=params.TEST_SIZE
        )

        return training_config

In [18]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
import numpy as np
import csv
import pandas as pd

In [19]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [22]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = load_model(self.config.updated_base_model_path)

    def train_test_generator(self):

        features = np.load(os.path.join(self.config.training_data,"features.npy"))
        labels= np.load(os.path.join(self.config.training_data,"labels.npy"))
        video_files_paths = np.load(os.path.join(self.config.training_data,"video_files_paths.npy"))

        one_hot_encoded_labels = to_categorical(labels)
        features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size=self.config.params_test_size,
                                                                            shuffle=True, random_state=42)

        return features_train, features_test, labels_train, labels_test

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def save_csv(path, data):
        with open(path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data.keys())
            writer.writerow(data.values())

    
    def train(self):
        features_train, features_test, labels_train, labels_test=self.train_test_generator()

        early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=5, min_lr=0.00005, verbose=1)

        model_checkpoint = ModelCheckpoint(self.config.model_checkpoints_path, save_best_only=True)
        
        self.model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

        model_history = self.model.fit(
            x=features_train, 
            y=labels_train, 
            epochs=self.config.params_epochs, 
            batch_size=self.config.params_batch_size, 
            shuffle=True, 
            validation_split=self.config.params_test_size, 
            callbacks=[early_stopping_callback, reduce_lr, model_checkpoint])
        
        model_evaluation_history = self.model.evaluate(features_test, labels_test)

        self.save_model(self.config.trained_model_path,self.model)
        self.save_model(self.config.prediction_model_path,self.model)
        pd.DataFrame(model_history.history).to_csv(self.config.model_history, index=False)
        pd.DataFrame(model_evaluation_history).to_csv(self.config.model_evaluation_history, index=False)

In [23]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train()
    
except Exception as e:
    raise e

[2024-03-31 18:48:27,563: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-31 18:48:27,565: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-31 18:48:27,566: INFO: common: created directory at: artifacts]
[2024-03-31 18:48:27,567: INFO: common: created directory at: artifacts\training]
Epoch 1/50
71/71 [==============================] - 11s 77ms/step - loss: 0.6965 - accuracy: 0.5326 - val_loss: 0.6838 - val_accuracy: 0.5079 - lr: 0.0100
Epoch 2/50
71/71 [==============================] - 4s 56ms/step - loss: 0.6771 - accuracy: 0.5538 - val_loss: 0.6556 - val_accuracy: 0.7143 - lr: 0.0100
Epoch 3/50
71/71 [==============================] - 4s 56ms/step - loss: 0.6348 - accuracy: 0.6596 - val_loss: 0.5738 - val_accuracy: 0.8413 - lr: 0.0100
Epoch 4/50
71/71 [==============================] - 4s 56ms/step - loss: 0.5203 - accuracy: 0.7743 - val_loss: 0.3598 - val_accuracy: 0.9683 - lr: 0.0100
Epoch 5/50
71/71 [==============================] - 4